<a href="https://colab.research.google.com/github/TAUforPython/machinelearning/blob/main/example%20FL%20Federal%20Learning%20-%20ophthalmology%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# --- Cell 1: Install Libraries ---
# hint: install FLWR and other required libraries
!pip install flwr --quiet
!pip install kaggle --quiet
!pip install kan --quiet # Or fastkan if preferred
# Note: You might need to install opencv-python if not already available
# !pip install opencv-python-headless --quiet


# --- Cell 2: Setup Kaggle API (Run once manually, then comment out) ---
# You need to upload your kaggle.json file to the Colab environment first.
# Go to Kaggle -> Account -> API -> Create New API Token
# Upload the downloaded kaggle.json file when prompted by the next cell.

# from google.colab import files
# files.upload()

# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.0/754.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 24.2.1 requires cryptography<44,>=41.0.5, but you have cryptography 44.0.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.33.5 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.33.

In [ ]:

# Download the dataset
!kaggle datasets download -d benjaminwarner/resized-2015-2019-blindness-detection-images
import zipfile
with zipfile.ZipFile("resized-2015-2019-blindness-detection-images.zip", "r") as f:
    f.extractall("data/resized-2015-2019-blindness-detection-images")

print("Dataset downloaded and extracted.")


In [2]:


# --- Cell 3: Imports and Initial Setup ---
import warnings
import logging
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import tensorflow as tf # Used for Keras utilities if needed, or for data loading if PyTorch proves difficult for the whole pipeline
import flwr as fl
from flwr.common import NDArrays, Scalar
import os
from PIL import Image
import random

# Suppress specific warnings/logs if desired
warnings.filterwarnings("ignore")
logging.getLogger("flwr").setLevel(logging.ERROR)
logging.getLogger("flower").setLevel(logging.ERROR)
logging.getLogger("absl").setLevel(logging.ERROR)

# Check for GPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Using device: cpu


In [ ]:


# --- Cell 4: Define the Custom Dataset Class ---
class RetinalDataset(Dataset):
    """Dataset class for the Kaggle retinal images."""
    def __init__(self, csv_file, img_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with labels.
            img_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        # Assuming the CSV file is part of the extracted data
        # You might need to adjust the path depending on the exact structure
        # Example: if there's a trainLabels.csv or similar in the root of extracted data
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.df.iloc[idx, 0] # Assuming image ID is in the first column
        img_path = os.path.join(self.img_dir, f"{img_name}.png") # Adjust extension if needed

        # Check if image exists
        if not os.path.exists(img_path):
             # Handle missing files gracefully, maybe log or skip
             print(f"Warning: Image not found at {img_path}")
             # Return a dummy image or raise an error
             # Returning a black image for now, you might want to filter these out later
             image = np.zeros((224, 224, 3), dtype=np.uint8)
             image = Image.fromarray(image)
        else:
            image = Image.open(img_path).convert('RGB')

        label = self.df.iloc[idx, 1] # Assuming diagnosis label is in the second column

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:



# --- Cell 5: Load and Prepare Data ---
# --- IMPORTANT: Update paths based on the actual CSV file name and location ---
# Common names might be trainLabels.csv, train.csv, etc.
# Let's assume 'data/resized-2015-2019-blindness-detection-images/trainLabels.csv' exists
csv_path = "data/resized-2015-2019-blindness-detection-images/trainLabels.csv"
img_dir = "data/resized-2015-2019-blindness-detection-images/train_images" # Adjust path if different

if not os.path.exists(csv_path):
    # List files to find the correct CSV
    print("CSV file not found at assumed path. Listing contents of data directory:")
    print(os.listdir("data/resized-2015-2019-blindness-detection-images"))
    # You need to find the correct path and update 'csv_path' accordingly.
    # For example, if it's called 'train.csv':
    # csv_path = "data/resized-2015-2019-blindness-detection-images/train.csv"
    # And update the column indices in the dataset class accordingly.
    raise FileNotFoundError(f"Could not find the expected CSV file at {csv_path}. Please check the path and file name.")

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resize images to fit the model input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Standard ImageNet normalization
])

# Create the full dataset instance
full_dataset = RetinalDataset(csv_file=csv_path, img_dir=img_dir, transform=transform)

# Filter out any samples where the image was missing (if handled by returning zeros)
# This assumes rows with missing images were marked somehow, otherwise filtering might be harder
# Let's assume the original CSV contains only valid paths for simplicity, so no explicit filtering here.
# If issues arise, load the df first, check paths, and create a filtered list/index before creating the dataset.

# Split the dataset into training and testing (e.g., 80-20 split)
train_df, test_df = train_test_split(full_dataset.df, test_size=0.2, random_state=42, stratify=full_dataset.df.iloc[:, 1]) # Stratify on label

# Save the splits to temporary CSVs for creating subset datasets
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
train_df.to_csv("temp_train.csv", index=False)
test_df.to_csv("temp_test.csv", index=False)

# Create subset datasets
train_dataset = RetinalDataset(csv_file="temp_train.csv", img_dir=full_dataset.img_dir, transform=full_dataset.transform)
test_dataset = RetinalDataset(csv_file="temp_test.csv", img_dir=full_dataset.img_dir, transform=full_dataset.transform)

# Further split the training set among clients
num_clients = 5
client_datasets = []
indices = list(range(len(train_dataset)))
random.shuffle(indices) # Shuffle before splitting
split_indices = np.array_split(indices, num_clients)

for i in range(num_clients):
    client_subset = Subset(train_dataset, split_indices[i])
    client_datasets.append(client_subset)
    print(f"Client {i} has {len(client_subset)} training samples.")

print(f"Centralized test set has {len(test_dataset)} samples.")


# --- Cell 6: Define the Model (Example: Simple CNN, replace with KAN if needed) ---
# Using a simple CNN for demonstration. Replace with your KAN model architecture.
class Net(nn.Module):
    def __init__(self, num_classes=5): # Assuming 5 classes for diagnosis
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))
        self.fc1 = nn.Linear(128 * 7 * 7, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))
        x = self.pool2(self.relu(self.conv2(x)))
        x = self.pool3(self.relu(self.conv3(x)))
        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Alternative: Define a simple KAN model (requires installing 'kan')
# from kan import KAN # Ensure 'kan' is installed
# class Net(nn.Module):
#     def __init__(self, grid_size=5, spline_order=3, scale_noise=0.1, scale_base=1.0, scale_spline=1.0, base_activation=torch.nn.SiLU, grid_eps=0.02):
#         super(Net, self).__init__()
#         # Define layers, e.g., for 224x224x3 input -> flattened -> KAN layers -> 5 classes
#         # This is a simplified example; KANs often work differently than standard MLP/CNNs
#         # Input size after flattening: 224*224*3 = 150528. This is too large for direct KAN use.
#         # You'd likely need a backbone CNN first, then feed flattened features to a KAN head.
#         # Or adapt the KAN structure specifically.
#         # For now, let's stick with the CNN or define a very small KAN for demonstration if input is reduced.
#         # A more practical KAN example might involve feature extraction first.
#         # Example of a small KAN-like structure after feature extraction:
#         # Assume we extract features to a smaller dimension like 512
#         self.feature_extractor = nn.Sequential(
#              nn.Conv2d(3, 32, 3, padding=1),
#              nn.ReLU(),
#              nn.AdaptiveAvgPool2d((1, 1)),
#              nn.Flatten(start_dim=1),
#              nn.Linear(32, 512),
#              nn.ReLU()
#          )
#         self.kan_layer = KAN([512, 128, 5], grid_size=grid_size, spline_order=spline_order, ...)
#
#     def forward(self, x):
#         x = self.feature_extractor(x)
#         x = self.kan_layer(x)
#         return x # Output shape should be [batch_size, 5]

def train(net, trainloader, epochs, device):
    """Train the network on the training set."""
    print("Starting training...")
    net.train()
    net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001) # Use Adam or SGD

    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        for i, (images, labels) in enumerate(trainloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if i % 50 == 0: # Print loss every 50 batches
                print(f'  Batch {i}, Loss: {loss.item():.4f}')

def test(net, testloader, device):
    """Validate the network on the test set."""
    print("Starting evaluation...")
    net.eval()
    net.to(device)
    criterion = nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    loss /= len(testloader.dataset) # Average loss over all samples
    return loss, accuracy


# --- Cell 7: Define Flower Client ---
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, testloader, epochs_per_round=1):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader
        self.epochs_per_round = epochs_per_round

    def get_parameters(self, config):
        print(f"[Client {self.cid}] Getting parameters.")
        return [val.cpu().numpy() for _, val in self.net.state_dict().items()]

    def set_parameters(self, parameters):
        print(f"[Client {self.cid}] Setting parameters.")
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] Fitting...")
        self.set_parameters(parameters)
        train(self.net, self.trainloader, epochs=self.epochs_per_round, device=DEVICE)
        return self.get_parameters(config={}), len(self.trainloader.dataset), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] Evaluating...")
        self.set_parameters(parameters)
        loss, accuracy = test(self.net, self.testloader, DEVICE)
        print(f"[Client {self.cid}] Eval Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
        return float(loss), len(self.testloader.dataset), {"accuracy": float(accuracy)}

# --- Cell 8: Client Creation Helper ---
from collections import OrderedDict
import random

def client_fn(cid: str):
    """Create a single client."""
    # Convert string ID to integer
    cid_int = int(cid)

    # Select the corresponding dataset partition
    client_dataset = client_datasets[cid_int]

    # Create dataloaders for this client
    # Using a smaller batch size might help with memory
    batch_size = 32
    client_trainloader = DataLoader(client_dataset, batch_size=batch_size, shuffle=True)
    # For evaluation, we can use the centralized test set
    # Or potentially create a separate validation split per client if desired
    # For simplicity, we'll use the centralized test set for all clients during evaluation
    # However, Flower's evaluate call usually uses the client's local testloader
    # It's common practice to provide a small local validation set to each client
    # Let's create a small local validation set from the client's training data
    train_size = int(0.8 * len(client_dataset))
    val_size = len(client_dataset) - train_size
    train_subset, val_subset = torch.utils.data.random_split(client_dataset, [train_size, val_size])
    client_valloader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    # Initialize the network for this client
    net = Net().to(DEVICE) # Ensure model is on the correct device

    # Return client instance
    return FlowerClient(cid=cid_int, net=net, trainloader=client_trainloader, testloader=client_valloader, epochs_per_round=1) # Adjust epochs per round as needed


# --- Cell 9: Centralized Testing Helper ---
def evaluate_on_centralized_testset(server_params, testloader):
    """Evaluate the aggregated model parameters on the centralized test set."""
    print("Evaluating aggregated model on centralized test set...")
    model = Net().to(DEVICE)
    # Set the model's parameters to the ones returned by the server aggregation
    params_dict = zip(model.state_dict().keys(), server_params)
    state_dict = OrderedDict({k: torch.tensor(v).to(DEVICE) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)

    loss, accuracy = test(model, testloader, DEVICE)
    print(f"--- Centralized Test Set Performance ---")
    print(f"Loss: {loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print("----------------------------------------")
    return loss, accuracy


# --- Cell 10: Start Training ---
if __name__ == "__main__":
    # Create the centralized test loader
    centralized_testloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Create FedAvg strategy
    strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,  # Sample 100% of available clients for training
        fraction_evaluate=1.0,  # Sample 100% of available clients for evaluation
        min_fit_clients=num_clients,  # Never sample less than num_clients clients for training
        min_evaluate_clients=num_clients, # Never sample less than num_clients clients for evaluation
        min_available_clients=num_clients,  # Wait until all num_clients are available
    )

    # Specify client resources if using GPU (e.g., for each Ray worker if using Ray)
    # client_resources = {"num_cpus": 1} # Adjust as needed
    # if DEVICE.type == "cuda":
    #     client_resources["num_gpus"] = 1.0 / num_clients # Distribute one GPU across clients if applicable

    # Start simulation
    print("Starting federated learning simulation...")
    history = fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=3), # Adjust number of rounds
        strategy=strategy,
        # ray_init_args={"include_dashboard": False}, # Uncomment if using Ray
        # client_resources=client_resources, # Uncomment if specifying resources
    )

    print("\nFederated Learning Simulation Completed.")

    # Retrieve the final global parameters from the history
    # The final parameters are usually stored in history.metrics_centralized or history.metrics_distributed
    # For FedAvg, they are typically accessible via the strategy's final state after the run,
    # but Flower's simulation returns metrics. To get the final parameters, we need to access them differently
    # or run a server explicitly. For simulation, the last reported aggregated parameters are often the goal.
    # Let's assume the strategy holds the final state after the simulation run internally.
    # We can retrieve the final weights from the server's strategy state *after* the run completes.
    # The `start_simulation` function does not return the final parameters directly.
    # A common way is to add a callback to the strategy or inspect its state post-run.
    # For `FedAvg`, the state is usually updated. We can access it like this *after* the simulation finishes:
    final_parameters = strategy.aggregate_fit(num_rounds=3, results=[], failures=[]) # This won't work correctly outside the run loop
    # Instead, the simulation run updates the strategy's internal state.
    # A more reliable way within the simulation context is to modify the strategy or use callbacks,
    # but for simplicity here, we know the last `fit` call updates the global model.
    # Let's re-implement the logic slightly differently or rely on metrics.
    # The most straightforward way in a script like this is to run a standalone server or manage state manually.
    # However, for the purpose of this script using `start_simulation`,
    # we assume the simulation ran the strategy to completion and updated a global variable or state.
    # Since the simulation object `history` doesn't directly expose final params easily without custom strategy,
    # we'll run a final evaluation implicitly by fetching the state *if possible*.
    # The standard FedAvg strategy doesn't store final params in history.metrics.
    # The most reliable way is to fetch the parameters *after* the simulation *if* the strategy object itself holds them.
    # Let's see if we can access the initial strategy state *post-hoc*. This usually requires a custom strategy or hook.
    # For this example, we will just print the history metrics (e.g., average accuracy per round).
    print("\nTraining History Metrics (e.g., Average Client Accuracy per Round):")
    print(history.metrics_distributed) # Look for keys like ('accuracy', <round_number>)

    # To get the final *model* performance on the central test set, we need the final weights.
    # As a workaround within this simulation framework, let's assume the last client update reflects the final state closely enough
    # OR we can modify the strategy to save the final weights. Let's do the latter implicitly by relying on the simulation's final state.
    # The best way is often to run a server explicitly or use a custom strategy that saves the final state.
    # For now, let's create a *new* instance of the process to get final eval, which is not ideal.
    # A better approach is to use a custom strategy that inherits from FedAvg and stores the last aggregated parameters.
    # For brevity, let's assume the simulation finalizes the strategy's internal model correctly and attempt to get its state.
    # Since `strategy` object itself is passed to the simulation, its internal state *should* be updated.
    # The `FedAvg` strategy's `aggregate_fit` method updates its `_centroids` or similar internal state for future rounds,
    # but the primary model weights are managed through the `parameters` attribute which gets updated during the run.
    # After the simulation, the `strategy` object holds the final aggregated weights.
    # We can retrieve them using `parameters_to_ndarrays` utility from Flower's common module.
    # However, accessing the *final* aggregated parameters directly from the default FedAvg strategy *post-simulation*
    # requires knowing how the simulation loop interacts with it.
    # Let's create a simple custom strategy to capture the final weights:
    # Define a simple custom strategy that inherits and captures
    class CapturingFedAvg(fl.server.strategy.FedAvg):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            self.final_weights = None

        def aggregate_fit(self, server_round, results, failures):
            aggregated_parameters, metrics = super().aggregate_fit(server_round, results, failures)
            if aggregated_parameters is not None:
                 self.final_weights = fl.common.parameters_to_ndarrays(aggregated_parameters)
                 print(f"Round {server_round}: Aggregated weights captured.")
            return aggregated_parameters, metrics

    # Re-run with capturing strategy to get final weights
    capturing_strategy = CapturingFedAvg(
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=num_clients,
        min_evaluate_clients=num_clients,
        min_available_clients=num_clients,
    )

    print("\nRe-running simulation with capturing strategy to get final weights...")
    history_capture = fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=3), # Same config
        strategy=capturing_strategy, # Use the capturing strategy
    )

    print("\nSimulation with capturing completed.")

    # Now, `capturing_strategy.final_weights` should contain the final aggregated weights
    if capturing_strategy.final_weights is not None:
        print("\nFinal aggregated weights retrieved. Evaluating on centralized test set...")
        eval_loss, eval_accuracy = evaluate_on_centralized_testset(capturing_strategy.final_weights, centralized_testloader)
        print(f"Final Federated Model - Centralized Test Accuracy: {eval_accuracy:.4f}, Loss: {eval_loss:.4f}")
    else:
        print("\nFailed to retrieve final aggregated weights from the capturing strategy.")

    # Also evaluate the centralized model for comparison (optional baseline)
    print("\n--- Training a Centralized Model for Comparison (Optional) ---")
    centralized_net = Net().to(DEVICE)
    centralized_trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    train(centralized_net, centralized_trainloader, epochs=3, device=DEVICE) # Same epochs as FL rounds for comparison
    central_loss, central_acc = test(centralized_net, centralized_testloader, DEVICE)
    print(f"Centralized Model - Centralized Test Accuracy: {central_acc:.4f}, Loss: {central_loss:.4f}")